In [1]:
#% pip install openpyxl
import os, re, configparser
import pandas as pd
import numpy as np
from globals import glob
from utilities_database import prepare_data, prepare_database, consult_database, clear_databases, retrieve_data, rename_index, get_date, get_sigma, rename_limits_table
from utilities_analysis import mean_calculator, limits_generator, ini_generator, RyR, z_score_filter, reset_df
from utilities_plotting import plot_scatter, plot_capability
import _db_tools as db

Database *input.db* found in: c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\input.db
Database *output.db* found in: c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\output.db


In [2]:
#Data extraction
table_names = ['VA_007_measurements_2023y_12m_21d_14h_05m_01s', 'VA_007_limits_2023y_12m_21d_14h_05m_01s']
MEAS = retrieve_data("input.db", table_names[0])
LIMITS = retrieve_data("input.db", table_names[1])

Table *VA_007_measurements_2023y_12m_21d_14h_05m_01s* retrieved succesfully.
Table *VA_007_limits_2023y_12m_21d_14h_05m_01s* retrieved succesfully.


In [3]:
#Calculate the means per nest.
means_df = mean_calculator(MEAS) #Global means version
    #means_df = mean_calculator(MEAS, glob.lenses_per_nest) #Positions mean version
means_df_format = rename_index(means_df)
means_df_format

,mean
Guia_Luz_Blanco_FB1_X,0.325980
Guia_Luz_Blanco_FB1_Y,0.348256
Guia_Luz_Blanco_FB2_X,0.325980
Guia_Luz_Blanco_FB2_Y,0.348256
Guia_Luz_Blanco_FB3_X,0.325980
Guia_Luz_Blanco_FB3_Y,0.348256
Guia_Luz_Blanco_FB4_X,0.325980
Guia_Luz_Blanco_FB4_Y,0.348256
Guia_Luz_Blanco_FB5_X,0.325980
Guia_Luz_Blanco_FB5_Y,0.348256


In [4]:
#Calculation of limits dataframe
new_limits = limits_generator(means_df)
limits_format = rename_index(new_limits)
limits_format

,LO_LIMIT,HI_LIMIT
Guia_Luz_Blanco_FB1_X,0.313480,0.338480
Guia_Luz_Blanco_FB1_Y,0.331756,0.364756
Guia_Luz_Blanco_FB2_X,0.313480,0.338480
Guia_Luz_Blanco_FB2_Y,0.331756,0.364756
Guia_Luz_Blanco_FB3_X,0.313480,0.338480
Guia_Luz_Blanco_FB3_Y,0.331756,0.364756
Guia_Luz_Blanco_FB4_X,0.313480,0.338480
Guia_Luz_Blanco_FB4_Y,0.331756,0.364756
Guia_Luz_Blanco_FB5_X,0.313480,0.338480
Guia_Luz_Blanco_FB5_Y,0.331756,0.364756


In [5]:
MEAS_format = rename_index(MEAS)
new_column_names = {old_col: f"test: {i+1}" for i, old_col in enumerate(MEAS_format.columns)}
MEAS_format.rename(columns=new_column_names, inplace=True)
output = pd.concat([MEAS_format, limits_format], axis=1) #Concatenates the measures and limits
output

,test: 1,test: 2,test: 3,test: 4,test: 5,test: 6,test: 7,test: 8,test: 9,test: 10,...,test: 23,test: 24,test: 25,test: 26,test: 27,test: 28,test: 29,test: 30,LO_LIMIT,HI_LIMIT
Guia_Luz_Blanco_FB1_X,0.3226,0.3225,0.3225,0.3225,0.3223,0.3223,0.3223,0.3221,0.3222,0.3222,...,0.3222,0.3221,0.3221,0.3222,0.3222,0.3221,0.3221,0.3222,0.313480,0.338480
Guia_Luz_Blanco_FB1_Y,0.3457,0.3455,0.3455,0.3455,0.3454,0.3454,0.3454,0.3451,0.3452,0.3452,...,0.3452,0.3452,0.3452,0.3452,0.3452,0.3452,0.3452,0.3452,0.331756,0.364756
Guia_Luz_Blanco_FB2_X,0.3274,0.3273,0.3272,0.3272,0.3271,0.3272,0.3271,0.3270,0.3270,0.3270,...,0.3268,0.3268,0.3268,0.3267,0.3269,0.3268,0.3268,0.3268,0.313480,0.338480
Guia_Luz_Blanco_FB2_Y,0.3490,0.3490,0.3489,0.3489,0.3489,0.3489,0.3488,0.3487,0.3487,0.3487,...,0.3486,0.3485,0.3485,0.3485,0.3486,0.3485,0.3485,0.3485,0.331756,0.364756
Guia_Luz_Blanco_FB3_X,0.3239,0.3239,0.3238,0.3238,0.3238,0.3238,0.3237,0.3236,0.3235,0.3237,...,0.3236,0.3236,0.3237,0.3237,0.3236,0.3236,0.3236,0.3236,0.313480,0.338480
Guia_Luz_Blanco_FB3_Y,0.3450,0.3450,0.3449,0.3449,0.3449,0.3449,0.3448,0.3447,0.3447,0.3448,...,0.3447,0.3447,0.3448,0.3448,0.3447,0.3447,0.3447,0.3447,0.331756,0.364756
Guia_Luz_Blanco_FB4_X,0.3268,0.3268,0.3267,0.3267,0.3268,0.3266,0.3267,0.3266,0.3266,0.3265,...,0.3265,0.3264,0.3264,0.3264,0.3264,0.3263,0.3264,0.3263,0.313480,0.338480
Guia_Luz_Blanco_FB4_Y,0.3502,0.3501,0.3501,0.3501,0.3501,0.3500,0.3500,0.3500,0.3500,0.3499,...,0.3498,0.3498,0.3498,0.3498,0.3497,0.3497,0.3498,0.3497,0.331756,0.364756
Guia_Luz_Blanco_FB5_X,0.3285,0.3284,0.3284,0.3283,0.3284,0.3283,0.3284,0.3282,0.3282,0.3282,...,0.3281,0.3280,0.3280,0.3280,0.3280,0.3280,0.3280,0.3280,0.313480,0.338480
Guia_Luz_Blanco_FB5_Y,0.3510,0.3508,0.3508,0.3508,0.3508,0.3508,0.3508,0.3507,0.3507,0.3506,...,0.3505,0.3505,0.3505,0.3504,0.3505,0.3504,0.3505,0.3504,0.331756,0.364756


In [6]:
##Data export and limit file generation
name = glob.tooling + "_" + get_date() + ".xlsx"
confirmation = input("Do you want to export the data to a new Target.xlsx file and generate a new limits ini file? (y/n): ").strip().lower()
if confirmation == 'y':
    output_filepath = os.path.join(os.path.abspath("../a2_output"), name)
    output.to_excel(output_filepath, index=True, header=True)
    #os.startfile('../a2_output/Target_with_limits.xlsx')
    ini_generator(new_limits, glob.lenses_per_nest, glob.nests_number)
    print(f'Data exported inside: {output_filepath}')
    print(f'Limits generated at: {os.path.split(output_filepath)[0]}')
else:
    print("Operation canceled.")

[LIMITS_GENERIC]
Guia_Luz_Blanco_FB1_X_MAX = 0.3385
Guia_Luz_Blanco_FB1_X_MIN = 0.3135
Guia_Luz_Blanco_FB1_Y_MAX = 0.3648
Guia_Luz_Blanco_FB1_Y_MIN = 0.3318
Guia_Luz_Blanco_FB2_X_MAX = 0.3385
...
Data exported inside: c:\Users\luciano.galan\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\a2_output\VA_007_2023y-12m-21d_15h-42m-18s.xlsx


In [7]:
##Update of the database table
name = re.sub(r'\W', '_', glob.tooling + "_" + get_date()) + "_limits&corrected"
confirmation = input("Do you want to export the new limits to your database (y/n): ").strip().lower()
if confirmation == 'y':
    prepare_database("output.db", new_limits, name) #Store a df for the limits inside the database of the project
    limits_file = rename_limits_table("output.db", name)
    consult_database("output.db")
else:
    print("Operation canceled.")

Operation canceled.
